In [ ]:
#Erstellen der Quelldatenbank 
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/DMA2022TeamA/main/quelldatenbank.py').text)

Mounted at /content/gdrive
Tabellen erfolgreich erstellt
Daten erfolgreich eingefügt


In [ ]:
#Initiierung Datawarehouse-Tabellen
Datawh_Tabellen = {}  

In [ ]:
#Definieren des Cursors zu der Datenbankdatei für das Datawarehouse
import sqlite3
cnx = sqlite3.connect('Datawarehouse.db')

In [ ]:
#Erstellen der zentralen Tabelle
Datawh_Tabellen['Zentrum'] = """
  create table if not exists Zentrum(
    PATIENT_ID VARCHAR REFERENCES dimPATIENT(ID),
    ENCOUNTER_ID VARCHAR REFERENCES dimENCOUNTERS(ID),
    OBSERVATION_ID VARCHAR REFERENCES dimOBSERVATIONS(ID)
  );
"""

#Einfügen in das Datawarehouse 
Zentrum = pd.read_sql_query("""
select encounters.patient_id as PATIENT_ID, encounters.id as ENCOUNTER_ID, observations.Id as OBSERVATION_ID from encounters 
left join observations on encounters.patient_id = observations.patient_id
where observations.code in ("8480-6","8462-4","9279-1","8867-4","3569","8310-5","1975-2","1920-8","1742-6","6768-6","33914-3","2885-2","3094-0","94531-1")
;""", conn
  )

Zentrum.to_sql(name = 'Zentrum', con=cnx, if_exists='append', index=False)

Zentrum.head(3)


,PATIENT_ID,ENCOUNTER_ID,OBSERVATION_ID
0,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,fd59905a-4ec8-62a6-d45d-d2a458d60111,131
1,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,fd59905a-4ec8-62a6-d45d-d2a458d60111,5
2,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,fd59905a-4ec8-62a6-d45d-d2a458d60111,14


In [ ]:
#Erstellen der Dimensionstabelle Observations
Datawh_Tabellen['dimObservations'] = """
  create table if not exists dimObservations(
    ID VARCHAR PRIMARY KEY,
    CODE VARCHAR,
    DESCRIPTION VARCHAR,
    VALUE VARCHAR,
    UNITS VARCHAR,
  );
"""

#Einfügen  in das Datawarehouse
dimObservations = pd.read_sql_query("""
Select ID, CODE, DESCRIPTION, VALUE, UNITS from observations
where observations.code in ("8480-6","8462-4","9279-1","8867-4","3569","8310-5","1975-2","1920-8","1742-6","6768-6","33914-3","2885-2","3094-0","94531-1")
;""", conn
  )

dimObservations.to_sql(name = 'dimObservations', con=cnx, if_exists='append', index=False)

dimObservations.head(3)


,Id,code,description,value,units
0,5,8462-4,Diastolic Blood Pressure,82.0,mm[Hg]
1,6,8480-6,Systolic Blood Pressure,110.0,mm[Hg]
2,7,8867-4,Heart rate,65.0,/min


In [ ]:
#Erstellen der Dimensionstabelle Encounters
Datawh_Tabellen['dimEncounters'] = """
  create table if not exists dimEncounters(
    ID VARCHAR PRIMARY KEY,
    START TIMESTAMP,
    STOP TIMESTAMP
    ENCOUNTERCLASS VARCHAR,
    CODE VARCHAR
    DESCRIPTION VARCHAR
  );
"""

#Einfügen  in das Datawarehouse
dimEncounters = pd.read_sql_query("""
Select ID, START, STOP, ENCOUNTERCLASS, CODE, DESCRIPTION from encounters
;""", conn
  )

dimEncounters.to_sql(name = 'dimEncounters', con=cnx, if_exists='append', index=False)

dimEncounters.head(3)

,id,start,stop,encounterclass,code,description
0,fd59905a-4ec8-62a6-d45d-d2a458d60111,1943-02-15T11:40:19Z,1943-02-15T11:55:19Z,ambulatory,185347001,Encounter for problem
1,de8159bf-90ae-c9a6-dd8d-f1de7b7972a7,1943-02-26T11:40:19Z,1943-02-26T11:55:19Z,ambulatory,185347001,Encounter for problem
2,6e0279b7-c477-a2c6-e75c-a911f03a264c,1946-01-11T11:40:19Z,1946-01-11T11:55:19Z,ambulatory,185345009,Encounter for symptom
